In [12]:
# Load libraries
import tensorflow as tf
from pandas import read_csv
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

# Load data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = read_csv(url, names=names, header=0)
dataset["class"] = dataset["class"].map({"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2})

# Split into train and test datasets
train, test = train_test_split(dataset, test_size=0.2)

# Extract labels
y_train = train.pop("class")
y_test = test.pop("class")

feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

# Make input function
def input_fn(features, labels, training=True, batch_size=256):
    # Convert input into dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # Shuffle dataset if in training
    if training:
        dataset = dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

# Build model with 2 hidden layers with 30 and 10 nodes each
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, hidden_units=[30, 10], n_classes=3)
# Train model for 5000 steps
classifier.train(input_fn = lambda:input_fn(train, y_train, training=True), steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/_w/lry6592x5hz5stxb0f_7bj5c0000gn/T/tmpdeuw5jmi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in f

In [13]:
# Evaluate Model
eval_result = classifier.evaluate(input_fn = lambda:input_fn(test, y_test, training=False))
clear_output()
print(eval_result["accuracy"])

1.0


In [15]:
SPECIES = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = train.keys()
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

Please type numeric values as prompted.
sepal-length: 6.7
sepal-width: 6.5
petal-length: 5.5
petal-width: 5.2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/_w/lry6592x5hz5stxb0f_7bj5c0000gn/T/tmpdeuw5jmi/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Iris-virginica" (59.9%)
